In [1]:
import sys
sys.path.insert(0, "..")
import snscrape.modules.twitter as sntwitter
from fastapi.encoders import jsonable_encoder
import json
import pandas as pd
import numpy as np
from pymongo import MongoClient
import datetime
import re
from config import conf
from utils import translate

client = MongoClient(conf['mongo_uri'])
db = client.politweet

In [2]:
tweets = db.tweets.find({'content_en':{'$exists':False}}, {'content':1})
for tweet in tweets:
    # print(tweet['_id'])
    content_en = translate(tweet['content'])
    db.tweets.update_one(tweet, {"$set":{"content_en": content_en}})

In [ ]:
conf = {
    'positions': {
        "AGIR-E": "centre-droit", # https://fr.wikipedia.org/wiki/Groupe_Agir_ensemble
        "DEM": "centre", # https://fr.wikipedia.org/wiki/Mouvement_d%C3%A9mocrate_(France)
        "DLF": "droite-plus", # https://fr.wikipedia.org/wiki/Debout_la_France
        "EDS": "centre-gauche", # https://fr.wikipedia.org/wiki/Groupe_%C3%89cologie_d%C3%A9mocratie_solidarit%C3%A9
        "FI": "gauche-plus", # https://fr.wikipedia.org/wiki/La_France_insoumise
        "GDR": "gauche-plus", # https://fr.wikipedia.org/wiki/Groupe_de_la_Gauche_d%C3%A9mocrate_et_r%C3%A9publicaine
        "GE": "centre-gauche", # https://fr.wikipedia.org/wiki/G%C3%A9n%C3%A9ration_%C3%A9cologie
        "LAREM": "centre", # https://fr.wikipedia.org/wiki/La_R%C3%A9publique_en_marche
        "LDS": "droite-plus", # https://fr.wikipedia.org/wiki/Ligue_du_Sud_(France)
        "LND": "gauche", # https://fr.wikipedia.org/wiki/Les_Nouveaux_D%C3%A9mocrates
        "LR": "droite", # https://fr.wikipedia.org/wiki/Les_R%C3%A9publicains
        "LT": "centre", # https://fr.wikipedia.org/wiki/Groupe_Libert%C3%A9s_et_territoires
        "RN": "droite-plus", # https://fr.wikipedia.org/wiki/Rassemblement_national
        "SOC": "gauche", # https://fr.wikipedia.org/wiki/Groupe_socialiste_(Assembl%C3%A9e_nationale)
        "UDI_I": "centre-droit", # https://fr.wikipedia.org/wiki/Union_des_d%C3%A9mocrates_et_ind%C3%A9pendants
    },
    "binary_positions": {
        "gauche": ["gauche", "centre-gauche", "gauche-plus"],
        "droite": ["droite", "centre-droit", "droite-plus"], 
    }
}




In [50]:
accounts = pd.DataFrame(list(db.accounts.find({}, {'_id':0, 'group_name':1, 'group':1})))
accounts

,group_name,group
0,Agir ensemble,AGIR-E
1,Agir ensemble,AGIR-E
2,Agir ensemble,AGIR-E
3,Agir ensemble,AGIR-E
4,Agir ensemble,AGIR-E
...,...,...
488,UDI et Indépendants,UDI_I
489,UDI et Indépendants,UDI_I
490,UDI et Indépendants,UDI_I
491,UDI et Indépendants,UDI_I


In [51]:
accounts = accounts.drop_duplicates(subset=['group'])

In [53]:
positions = {
    "AGIR-E": "centre-droit", # https://fr.wikipedia.org/wiki/Groupe_Agir_ensemble
    "DEM": "centre", # https://fr.wikipedia.org/wiki/Mouvement_d%C3%A9mocrate_(France)
    "DLF": "droite-plus", # https://fr.wikipedia.org/wiki/Debout_la_France
    "EDS": "centre-gauche", # https://fr.wikipedia.org/wiki/Groupe_%C3%89cologie_d%C3%A9mocratie_solidarit%C3%A9
    "FI": "gauche-plus", # https://fr.wikipedia.org/wiki/La_France_insoumise
    "GDR": "gauche-plus", # https://fr.wikipedia.org/wiki/Groupe_de_la_Gauche_d%C3%A9mocrate_et_r%C3%A9publicaine
    "GE": "centre-gauche", # https://fr.wikipedia.org/wiki/G%C3%A9n%C3%A9ration_%C3%A9cologie
    "LAREM": "centre", # https://fr.wikipedia.org/wiki/La_R%C3%A9publique_en_marche
    "LDS": "droite_plus", # https://fr.wikipedia.org/wiki/Ligue_du_Sud_(France)
    "LND": "gauche", # https://fr.wikipedia.org/wiki/Les_Nouveaux_D%C3%A9mocrates
    "LR": "droite", # https://fr.wikipedia.org/wiki/Les_R%C3%A9publicains
    "LT": "centre", # https://fr.wikipedia.org/wiki/Groupe_Libert%C3%A9s_et_territoires
    "RN": "droite-plus", # https://fr.wikipedia.org/wiki/Rassemblement_national
    "SOC": "gauche", # https://fr.wikipedia.org/wiki/Groupe_socialiste_(Assembl%C3%A9e_nationale)
    "UDI_I": "centre-droit", # https://fr.wikipedia.org/wiki/Union_des_d%C3%A9mocrates_et_ind%C3%A9pendants
}

,group_name,group
0,Agir ensemble,AGIR-E
20,Mouvement Démocrate (MoDem) et Démocrates appa...,DEM
66,Debout la France,DLF
68,Écologie démocratie solidarité,EDS
73,La France insoumise,FI
90,Gauche démocrate et républicaine,GDR
98,Génération Écologie,GE
100,La République en Marche,LAREM
351,Ligue du Sud,LDS
352,Les Nouveaux Démocrates,LND


In [58]:
groups = accounts.to_dict('records')
# groups
for group in groups:
    print("* **{}**: {}\\".format(group["group"], group["group_name"]))

* **AGIR-E**: Agir ensemble\
* **DEM**: Mouvement Démocrate (MoDem) et Démocrates apparentés\
* **DLF**: Debout la France\
* **EDS**: Écologie démocratie solidarité\
* **FI**: La France insoumise\
* **GDR**: Gauche démocrate et républicaine\
* **GE**: Génération Écologie\
* **LAREM**: La République en Marche\
* **LDS**: Ligue du Sud\
* **LND**: Les Nouveaux Démocrates\
* **LR**: Les Républicains\
* **LT**: Libertés et Territoires\
* **RN**: Rassemblement National\
* **SOC**: Socialistes et apparentés\
* **UDI_I**: UDI et Indépendants\


In [48]:
import random
from IPython.display import display, Markdown

def md(input):
    return display(Markdown(input))

def get_random_color():
    return "#"+''.join([random.choice('0123456789ABCDEF') for _ in range(6)])

for group in groups:
    group['color'] = get_random_color()
    print(group['color'])
    md("<span style='color:white;background :{};padding: 5px'>{}</span>".format(group['color'], group['group']))
print({group['group']:group['color'] for group in groups})

#5B17E3


<span style='color:white;background :#5B17E3;padding: 5px'>AGIR-E</span>

#415291


<span style='color:white;background :#415291;padding: 5px'>DEM</span>

#328855


...

#7C6437


<span style='color:white;background :#7C6437;padding: 5px'>SOC</span>

#9C7621


<span style='color:white;background :#9C7621;padding: 5px'>UDI_I</span>

{'AGIR-E': '#5B17E3', 'DEM': '#415291', 'DLF': '#328855', 'EDS': '#BA7096', 'FI': '#626539', 'GDR': '#99A8F7', 'GE': '#52B62B', 'LAREM': '#1484CB', 'LDS': '#248913', 'LND': '#64AE81', 'LR': '#BA2261', 'LT': '#14C55C', 'RN': '#4E1D74', 'SOC': '#7C6437', 'UDI_I': '#9C7621'}


In [49]:
colors_set = {'AGIR-E': '#5B17E3', 'DEM': '#415291', 'DLF': '#328855', 'EDS': '#BA7096', 'FI': '#626539', 'GDR': '#99A8F7', 'GE': '#52B62B', 'LAREM': '#1484CB', 'LDS': '#248913', 'LND': '#64AE81', 'LR': '#BA2261', 'LT': '#14C55C', 'RN': '#4E1D74', 'SOC': '#7C6437', 'UDI_I': '#9C7621'}

def get_css(colors_set):
    for group,color in colors_set.items():
        print(".label-{}{{background-color: {};}}".format(group,color))

get_css(colors_set)


.label-AGIR-E{background-color: #5B17E3;}
.label-DEM{background-color: #415291;}
.label-DLF{background-color: #328855;}
.label-EDS{background-color: #BA7096;}
.label-FI{background-color: #626539;}
.label-GDR{background-color: #99A8F7;}
.label-GE{background-color: #52B62B;}
.label-LAREM{background-color: #1484CB;}
.label-LDS{background-color: #248913;}
.label-LND{background-color: #64AE81;}
.label-LR{background-color: #BA2261;}
.label-LT{background-color: #14C55C;}
.label-RN{background-color: #4E1D74;}
.label-SOC{background-color: #7C6437;}
.label-UDI_I{background-color: #9C7621;}
